In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torchvision import transforms
import pandas as pd
from PIL import Image
import math

In [12]:

LR = 0.01
BATCH_SIZE = 64
EPOCH = 1000

DATA_FOLDER = 'set00'

In [13]:
def getData(mode):
    if mode == 'train':
        
        img = pd.read_csv(DATA_FOLDER + '_train_img.csv')
        label = pd.read_csv(DATA_FOLDER + '_train_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)
    else:
        img = pd.read_csv(DATA_FOLDER + '_test_img.csv')
        label = pd.read_csv(DATA_FOLDER + '_test_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)


class Loader(Data.Dataset):
    def __init__(self, root, mode):
        """
        Args:
            root (string): Root path of the dataset.
            mode : Indicate procedure status(training or testing)

            self.img_name (string list): String list that store all image names.
            self.label (int or float list): Numerical list that store all ground truth label values.
        """
        self.root = root
        self.img_name, self.label = getData(mode)
        self.mode = mode
        print("> Found %d images..." % (len(self.img_name)))
        

    def __len__(self):
        #------------return the size of dataset
        return len(self.img_name)

    def __getitem__(self, index):
        #-------------Get the image path from 'self.img_name' and load it.
                  
        path = self.root + self.img_name[index] + '.png'
        img = Image.open(path)
        img_as_img = img.resize(( 160, 120),Image.ANTIALIAS)
        
        #-------------Get the ground truth label from self.label"""
        label = torch.from_numpy(self.label)[index]
        
        #-------------Transform the .jpeg rgb images
        if self.mode == 'train':
            transform1 = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(degrees=(-45,45), resample=False, expand=False, center=None),
                transforms.ColorJitter(contrast=(0,1)),
                transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
                ]
            )
        else:
            transform1 = transforms.Compose([
                transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
                ]
            )
        img_tran = transform1(img_as_img)
                
        #-------------Return processed image and label
        return img_tran, label

In [14]:
# Dataloader

train_data=Loader('./'+ DATA_FOLDER +'/','train')
test_data=Loader('./'+ DATA_FOLDER +'/','test')
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE)
test_loader = Data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE)
print (train_loader)
print (test_loader)

> Found 2339 images...
> Found 259 images...


In [23]:

class DeepConvNet(nn.Module):
    def __init__(self):
        super(DeepConvNet, self).__init__()
    
        #####---1
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0, bias=False)
        self.batchnorm1 = nn.BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.norm1 = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=1.0)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        #####
        
        #####---2
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=(2, 2) ,bias=True, groups=2)
        self.batchnorm2 = nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.norm2 = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=1.0)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        #####
        
        #####---3
        self.conv3 = nn.Conv2d(256, 384, kernel_size=(3, 3), padding=(1, 1), bias=False)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=(3, 3), padding=(1, 1), bias=True, groups=2)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=(3, 3), padding=(1, 1), bias=True, groups=2)
        self.pool5 = nn.MaxPool2d(kernel_size=3 , stride=2, padding=0)
        #####
        
        self.fc6 = nn.Linear(in_features=1536, out_features=4096, bias=True)
        self.fc7 = nn.Linear(in_features=4096, out_features=4096, bias=True)
        self.fc8 = nn.Linear(in_features=4096, out_features=256, bias=False)
        self.fc9 = nn.Linear(in_features=256, out_features=2, bias=True)
        
    def forward(self, x):
        
        ###-----1------
        out = self.conv1(x)
        #out = self.batchnorm1(out)
        out = F.relu(out)
        out = self.pool1(out)        
        out = self.norm1(out)
        #out = F.dropout(out, p=0.5)
        
        ###-----2------
        out = self.conv2(out)
        #out = self.batchnorm2(out)
        out = F.relu(out)
        out = self.pool2(out)
        out = self.norm2(out)
        #out = F.dropout(out, p=0.5)
        
        ###-----3------
        out = self.conv3(out)
        out = F.relu(out)
        ###-----4------
        out = self.conv4(out)
        out = F.relu(out)
        
        out = self.conv5(out)
        out = F.relu(out)
        out = self.pool5(out)
        
        out = out.view(out.size(0), -1) #flatten
        out = self.fc6(out)
        out = F.relu(out)
        out = F.dropout(out, p=0.5)
        
        out = self.fc7(out)
        out = F.relu(out)
        out = F.dropout(out, p=0.5)
      
        out = self.fc8(out)
        out = F.relu(out)
        out = F.dropout(out, p=0.5)
        
        out = self.fc9(out)
        out = torch.sigmoid(out)
        
        
        return out

In [24]:
def train_DeepConv(epoch):
    model_DeepConvNet.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.to(device=device, dtype=torch.float), target.to(device)

        optimizer.zero_grad()
        output = model_DeepConvNet(data)
        target = target.float()
        loss_d = Loss(output[0], target[0])
        loss_phi = Loss(output[1], target[1])
        loss = loss_d + loss_phi
        loss.backward()
        optimizer.step()
    TRAIN_LOSS.append(round((loss.data).cpu().numpy().tolist(),6))
   
    print('Train Epoch: {} \t Loss_d: {} Loss_phi:{}'.format(
            epoch, loss_d.data , loss_phi.data))

In [25]:
def test_DeepConv(epoch):
    model_DeepConvNet.eval()
    test_loss_d = 0.0
    test_loss_phi = 0.0
    
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        if torch.cuda.is_available():
            data, target = data.to(device=device, dtype=torch.float), target.to(device)
        with torch.no_grad():
            output = model_DeepConvNet(data)
        target = target.float()
        test_loss_d = Loss(output[0], target[0])
        test_loss_phi = Loss(output[1], target[1])
        
    print('---------------Test set: Average loss d:{}\t loss phi:{}-------------------\n'
          .format(test_loss_d,test_loss_phi))

In [26]:
model_DeepConvNet = DeepConvNet().cuda(0)

if torch.cuda.is_available():
    device = torch.device('cuda')
    model_DeepConvNet.to(device) 
#------------define optimizer/loss function
Loss = nn.MSELoss(reduction='mean')    
optimizer = torch.optim.SGD(model_DeepConvNet.parameters(), lr=LR, momentum=0.9, 
                            dampening=0, weight_decay=0.0005, nesterov=False)
print(model_DeepConvNet)

DeepConvNet(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), bias=False)
  (batchnorm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm1): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=2)
  (batchnorm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
  (pool5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilati

In [ ]:
#---------------Train-----------------
for epoch in range(1, EPOCH + 1):
    INDEX.append(epoch)
    train_DeepConv(epoch)
    test_DeepConv(epoch)
    if (epoch%100==0):
        PATH =str(epoch)+'model.pkl'
        torch.save(model_DeepConvNet.state_dict(), PATH)
    '''
    savefilename = str(epoch)+'.tar'
    torch.save({
            'epoch': epoch,
            'state_dict': model_EEGNet.state_dict(),
        }, savefilename)
    '''


Train Epoch: 1 	 Loss_d: 0.17558278143405914 Loss_phi:0.09272637218236923
---------------Test set: Average loss d:0.2634773254394531	 loss phi:0.22268258035182953-------------------

Train Epoch: 2 	 Loss_d: 0.1837119311094284 Loss_phi:0.08929529041051865
---------------Test set: Average loss d:0.14850912988185883	 loss phi:0.12020845711231232-------------------

Train Epoch: 3 	 Loss_d: 0.21199269592761993 Loss_phi:0.10472551733255386
---------------Test set: Average loss d:0.09001616388559341	 loss phi:0.07246122509241104-------------------

Train Epoch: 4 	 Loss_d: 0.2673106789588928 Loss_phi:0.14970512688159943
---------------Test set: Average loss d:0.0608973503112793	 loss phi:0.034182358533144-------------------

Train Epoch: 5 	 Loss_d: 0.2757493555545807 Loss_phi:0.1578560769557953
---------------Test set: Average loss d:0.03923690319061279	 loss phi:0.030616292729973793-------------------

Train Epoch: 6 	 Loss_d: 0.2769702672958374 Loss_phi:0.16491112112998962
--------------

---------------Test set: Average loss d:0.03868221491575241	 loss phi:0.03000873327255249-------------------

Train Epoch: 46 	 Loss_d: 0.2796795964241028 Loss_phi:0.1651197373867035
---------------Test set: Average loss d:0.0386832021176815	 loss phi:0.030003421008586884-------------------

Train Epoch: 47 	 Loss_d: 0.27955788373947144 Loss_phi:0.1651199460029602
---------------Test set: Average loss d:0.0386803075671196	 loss phi:0.02999044954776764-------------------

Train Epoch: 48 	 Loss_d: 0.27882111072540283 Loss_phi:0.16511417925357819
---------------Test set: Average loss d:0.03869400545954704	 loss phi:0.029990561306476593-------------------

Train Epoch: 49 	 Loss_d: 0.2796659469604492 Loss_phi:0.16508431732654572
---------------Test set: Average loss d:0.03868115693330765	 loss phi:0.029999498277902603-------------------

Train Epoch: 50 	 Loss_d: 0.279621958732605 Loss_phi:0.16512008011341095
---------------Test set: Average loss d:0.038681380450725555	 loss phi:0.0299907

Train Epoch: 90 	 Loss_d: 0.2783358097076416 Loss_phi:0.16477864980697632
---------------Test set: Average loss d:0.038681771606206894	 loss phi:0.03000093810260296-------------------

Train Epoch: 91 	 Loss_d: 0.279507040977478 Loss_phi:0.1651025414466858
---------------Test set: Average loss d:0.0386890210211277	 loss phi:0.029990551993250847-------------------

Train Epoch: 92 	 Loss_d: 0.27962401509284973 Loss_phi:0.16509725153446198
---------------Test set: Average loss d:0.03869175165891647	 loss phi:0.02999105304479599-------------------

Train Epoch: 93 	 Loss_d: 0.27973949909210205 Loss_phi:0.1649307906627655
---------------Test set: Average loss d:0.03868034854531288	 loss phi:0.029990535229444504-------------------

Train Epoch: 94 	 Loss_d: 0.27938494086265564 Loss_phi:0.16506506502628326
---------------Test set: Average loss d:0.03869378939270973	 loss phi:0.02999028004705906-------------------

Train Epoch: 95 	 Loss_d: 0.2758478820323944 Loss_phi:0.1649831086397171
-----